<h1> INTER_INTER-CODER ANALYSIS </h1>

### IMPORTS

In [ ]:
import os
import time
import pandas as pd
import json
import ast
import pprint as pp
import ollama

### MODEL FILES

In [ ]:
#array of test LLMs
model_arr = [
    'llama3:70b',
    'phi3:14b',
    'mistral:7b',
    'gemma2:27b'
]

### EVALUATE
Make sure ollama server is running on the same node

In [ ]:
#for each model
for m in model_arr:  
   #create system prompt  
   modelfile='''
   FROM {}
   SYSTEM You are an impartial research assistant evaluating Twitter data collected during the COVID-19 pandemic. The tweets may be serious, humorous, or have nothing to do with COVID-19. 
   Please take into account links, emojis, hashtags, etc. Please respond with only two items: an integer (0 for 'not related to mask wearing',1 for 'anti-mask', 2 for 'pro-mask', 3 for neutral/uncertain) and a single sentence explaining your rationale. 
   Format your response as well-formed JSON as follows: {{"result":"1","rationale":"the individual hates mask-wearing"}}
   '''.format(m)
   #create model based on system prompt
   ollama.create(model='{}-crisis-1'.format(m), modelfile=modelfile)

In [ ]:
#read in tweet texts
df = pd.read_csv('data/mask_tweets.csv',index_col=0)
df['full_text']

0       Finally a mask that fits my true personality!!...
1       People screaming that we need to reopen school...
2       Rosie the Riveter going strong at 94🇺🇸 https:/...
3       So what happens when the vaccine comes? The an...
4       @Walsh_PT I was just saying to my wife that th...
                              ...                        
2576    @NoNameCalling1 @mcfunny @P8R1OT “You mask wea...
2577    @NoNameCalling1 @mcfunny @P8R1OT But let’s set...
2578    @NoNameCalling1 @JaneEOpie @mcfunny @TheBaba45...
2579    Dear @RepThomasMassie John Locke 'said that a ...
2580    And there you have it.\n@RepThomasMassie can y...
Name: full_text, Length: 2581, dtype: object

In [ ]:
#select the llm under test
#!!!!!iterate for each model, n times
test_model = model_arr[3]
#output data
output_dict = []
#for each tweet text
for row in df.iterrows():
    #create buffer
    buffer_dict = {}
    #note start time
    start_time = time.time()
    #query ollama for response
    response = ollama.chat(model='{}-crisis-1'.format(test_model), messages=[
      {
          'role': 'user',
          'content': 'What is the following Twitter user\'s attitude towards COVID-19 mask wearing?: {}'.format(row[1]["full_text"]),
      },
    ])
    #parse response as json and append to buffer
    #if parsing error, log as '-1' and keep the text
    try:
        buffer_dict = ast.literal_eval(response['message']['content'])
    except:
        buffer_dict = {"result":"-1","rationale":response['message']['content']}
    #note end time
    stop_time = time.time()
    #append original tweet information + evaluation time to buffer
    buffer_dict["tweet_id"] = row[1]["id"]
    buffer_dict["full_text"] = row[1]["full_text"]
    buffer_dict["eval_time"] = stop_time - start_time
    #append buffer to output
    output_dict.append(buffer_dict)
    #for testing
    #pp.pprint(buffer_dict)
#generate output from output dict and save
output_df = pd.DataFrame().from_dict(output_dict)
output_df.to_csv('data/output/{}/{}_crisis_1_{}.csv'.format(test_model,test_model,time.time()))
output_df